In [1]:
import re
import pandas as pd
import numpy as np

from nltk import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, GRU, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences

from transformers import BertTokenizerFast, TFBertModel

from scripts.word_embeddings import load_embedding_weights

2022-06-24 11:13:21.047386: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-06-24 11:13:21.047428: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (petar-X580VD): /proc/driver/nvidia/version does not exist
2022-06-24 11:13:21.048151: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Utility functions

In [2]:
def create_vocabulary(sentences_tokens):
    voc = set()
    
    voc.update(['<PAD>'])
    
    for tokens in sentences_tokens:
        voc.update(tokens)
        
    voc = list(voc)
        
    word_to_id = {word: index for word, index in zip(voc, range(len(voc)))}
    id_to_word = {index: word for word, index in zip(voc, range(len(voc)))}
    
#     print(word_to_id['<PAD>'])
    
    return voc, word_to_id, id_to_word

In [3]:
def clean(word):
    return re.sub(r'[\-\(\)\?\*\\\ \'\"\.%\^,:<>|;]', '', word.lower())

In [4]:
path = '/home/petar/Fakultet/Semester 7/NLP/Datasets/fake_news/'

In [5]:
train = pd.read_csv(path + 'full_train_df.csv', index_col=0)
test = pd.read_csv(path + 'full_test_df.csv', index_col=0)

# Preprocessing

In [6]:
train['tweet_tokens'] = train['tweet'].apply(lambda sentence: sentence.split(' '))
test['tweet_tokens'] = test['tweet'].apply(lambda sentence: sentence.split(' '))

In [7]:
train['tweet_tokens'] = train['tweet_tokens'].apply(lambda sentence: [clean(word) for word in sentence])
test['tweet_tokens'] = test['tweet_tokens'].apply(lambda sentence: [clean(word) for word in sentence])

In [8]:
tmp = train['tweet_tokens'].append(test['tweet_tokens'])

/tmp/ipykernel_9120/1723939186.py:1: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp = train['tweet_tokens'].append(test['tweet_tokens'])


In [9]:
tmp.index = range(len(tmp))

In [10]:
voc, word_to_id, id_to_word = create_vocabulary(tmp)

In [11]:
embeddings = load_embedding_weights(voc, 50, 'glove')

In [12]:
train['tweet_indices'] = train['tweet_tokens'].apply(lambda sentence_tokens: [word_to_id[token] for token in sentence_tokens])
test['tweet_indices'] = test['tweet_tokens'].apply(lambda sentence_tokens: [word_to_id[token] for token in sentence_tokens])

In [13]:
x = train['tweet_indices'].copy()
y = train['label'].copy()
# y = y.apply(lambda x: [x])

In [14]:
padded_x = pad_sequences(x, value=word_to_id['<PAD>'], maxlen=100)

In [15]:
padded_x = pd.Series(padded_x.tolist())

In [16]:
# padded_y = pad_sequences(y, value=word_to_id['<PAD>'], maxlen=100)

In [17]:
# padded_y = pd.Series(padded_y.tolist())

In [16]:
x_train, x_test, y_train, y_test = train_test_split(padded_x, y, test_size=.1, random_state=0, shuffle=True, stratify=y)

In [17]:
x_train = x_train.append(x_test)
y_train = y_train.append(y_test)

/tmp/ipykernel_9120/4134551260.py:1: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_train = x_train.append(x_test)
/tmp/ipykernel_9120/4134551260.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_train = y_train.append(y_test)


In [18]:
x = test['tweet_indices'].copy()
y = test['label'].copy()
# y = y.apply(lambda x: [x])

In [19]:
padded_x = pad_sequences(x, value=word_to_id['<PAD>'], maxlen=100)

In [20]:
padded_x = pd.Series(padded_x.tolist())

In [23]:
# padded_y = pad_sequences(y, value=word_to_id['<PAD>'], maxlen=100)

In [24]:
# padded_y = pd.Series(padded_y.tolist())

In [27]:
x_train, x_test, y_train, y_test = train_test_split(padded_x, y, test_size=.1, random_state=0, shuffle=True, stratify=y)

In [28]:
x_test = x_train.append(x_test)
y_test = y_train.append(y_test)

/tmp/ipykernel_9120/1631974232.py:1: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_test = x_train.append(x_test)
/tmp/ipykernel_9120/1631974232.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_test = y_train.append(y_test)


In [29]:
embeddings = embeddings.astype(np.float32)

In [39]:
x_train_tmp = x_train.to_list()
y_train_tmp = y_train.to_list()

x_test_tmp = x_test.to_list()
y_test_tmp = y_test.tolist()

# DNN models

# Model1

In [33]:
model1 = Sequential()
# model1.add(Input(shape=(100,)))
model1.add(Embedding(input_dim=len(voc), output_dim=50, weights=[embeddings], trainable=True))
# model1.add(LSTM(64, return_sequences=True))
# model1.add(LSTM(64, return_sequences=True))
# model1.add(LSTM(64))
model1.add(LSTM(8))
model1.add(Dense(1, activation='sigmoid'))

model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model1.summary()

2022-06-20 13:45:30.710729: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-20 13:45:30.717889: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-20 13:45:30.718175: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-20 13:45:30.726985: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          2948450   
                                                                 
 lstm (LSTM)                 (None, 8)                 1888      
                                                                 
 dense (Dense)               (None, 1)                 9         
                                                                 
Total params: 2,950,347
Trainable params: 2,950,347
Non-trainable params: 0
_________________________________________________________________


In [34]:
model1.fit(x_train_tmp, y_train_tmp, epochs=5, batch_size=32, verbose=2, use_multiprocessing=True)

Epoch 1/5


2022-06-20 13:45:59.169297: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101


563/563 - 22s - loss: 0.6656 - accuracy: 0.5881 - 22s/epoch - 40ms/step
Epoch 2/5
563/563 - 4s - loss: 0.5724 - accuracy: 0.7055 - 4s/epoch - 8ms/step
Epoch 3/5
563/563 - 4s - loss: 0.4456 - accuracy: 0.7967 - 4s/epoch - 8ms/step
Epoch 4/5
563/563 - 5s - loss: 0.3242 - accuracy: 0.8654 - 5s/epoch - 8ms/step
Epoch 5/5
563/563 - 4s - loss: 0.2248 - accuracy: 0.9153 - 4s/epoch - 8ms/step


In [64]:
y_pred = model1.predict(x_test_tmp)
y_pred_tmp = np.round(y_pred)
y_pred_tmp = np.hstack(y_pred_tmp)
y_pred_tmp = y_pred_tmp.astype(np.int32)
print(classification_report(y_test, y_pred_tmp))

625/625 [==============================] - 2s 2ms/step
              precision    recall  f1-score   support

           0       0.93      0.92      0.93      9999
           1       0.92      0.93      0.93     10000

    accuracy                           0.93     19999
   macro avg       0.93      0.93      0.93     19999
weighted avg       0.93      0.93      0.93     19999



In [74]:
model1.save('model1/')

INFO:tensorflow:Assets written to: model1/assets


INFO:tensorflow:Assets written to: model1/assets


In [ ]:
# model1 = tf.keras.models.load_model('model1/')

# Model2

In [58]:
model2 = Sequential()

model2.add(Embedding(input_dim=len(voc), output_dim=50, weights=[embeddings], trainable=True))
model2.add(Bidirectional(LSTM(8)))
model2.add(Dense(1, activation='sigmoid'))

model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 50)          2948450   
                                                                 
 bidirectional (Bidirectiona  (None, 16)               3776      
 l)                                                              
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 2,952,243
Trainable params: 2,952,243
Non-trainable params: 0
_________________________________________________________________


In [59]:
model2.fit(x_train_tmp, y_train_tmp, epochs=5, batch_size=32, verbose=2, use_multiprocessing=True)

Epoch 1/5
563/563 - 9s - loss: 0.6672 - accuracy: 0.5878 - 9s/epoch - 16ms/step
Epoch 2/5
563/563 - 6s - loss: 0.5676 - accuracy: 0.7083 - 6s/epoch - 12ms/step
Epoch 3/5
563/563 - 6s - loss: 0.4354 - accuracy: 0.8035 - 6s/epoch - 11ms/step
Epoch 4/5
563/563 - 6s - loss: 0.3175 - accuracy: 0.8694 - 6s/epoch - 11ms/step
Epoch 5/5
563/563 - 6s - loss: 0.2288 - accuracy: 0.9145 - 6s/epoch - 11ms/step


In [60]:
y_pred = model2.predict(x_test_tmp)
y_pred_tmp = np.round(y_pred)
y_pred_tmp = np.hstack(y_pred_tmp)
y_pred_tmp = y_pred_tmp.astype(np.int32)
print(classification_report(y_test, y_pred_tmp))

625/625 [==============================] - 3s 3ms/step
              precision    recall  f1-score   support

           0       0.94      0.91      0.93      9999
           1       0.91      0.94      0.93     10000

    accuracy                           0.93     19999
   macro avg       0.93      0.93      0.93     19999
weighted avg       0.93      0.93      0.93     19999



In [73]:
model2.save('model2/')

INFO:tensorflow:Assets written to: model2/assets


INFO:tensorflow:Assets written to: model2/assets


In [ ]:
# model2 = tf.keras.models.load_model('model2/')

# Model3

In [65]:
model3 = Sequential()
model3.add(Embedding(input_dim=len(voc), output_dim=50, weights=[embeddings], trainable=True))
model3.add(LSTM(64, return_sequences=True))
model3.add(LSTM(64, return_sequences=True))
model3.add(LSTM(64))

model3.add(Dense(1, activation='sigmoid'))

model3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 50)          2948450   
                                                                 
 lstm_2 (LSTM)               (None, None, 64)          29440     
                                                                 
 lstm_3 (LSTM)               (None, None, 64)          33024     
                                                                 
 lstm_4 (LSTM)               (None, 64)                33024     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 3,044,003
Trainable params: 3,044,003
Non-trainable params: 0
_________________________________________________________________


In [66]:
model3.fit(x_train_tmp, y_train_tmp, epochs=5, batch_size=32, verbose=2, use_multiprocessing=True)

Epoch 1/5
563/563 - 13s - loss: 0.6521 - accuracy: 0.6081 - 13s/epoch - 23ms/step
Epoch 2/5
563/563 - 10s - loss: 0.5360 - accuracy: 0.7247 - 10s/epoch - 18ms/step
Epoch 3/5
563/563 - 9s - loss: 0.4027 - accuracy: 0.8134 - 9s/epoch - 16ms/step
Epoch 4/5
563/563 - 9s - loss: 0.2838 - accuracy: 0.8772 - 9s/epoch - 16ms/step
Epoch 5/5
563/563 - 9s - loss: 0.1804 - accuracy: 0.9274 - 9s/epoch - 16ms/step


In [67]:
y_pred = model3.predict(x_test_tmp)
y_pred_tmp = np.round(y_pred)
y_pred_tmp = np.hstack(y_pred_tmp)
y_pred_tmp = y_pred_tmp.astype(np.int32)
print(classification_report(y_test, y_pred_tmp))

625/625 [==============================] - 4s 6ms/step
              precision    recall  f1-score   support

           0       0.96      0.91      0.94      9999
           1       0.91      0.97      0.94     10000

    accuracy                           0.94     19999
   macro avg       0.94      0.94      0.94     19999
weighted avg       0.94      0.94      0.94     19999



In [72]:
model3.save('model3/')

INFO:tensorflow:Assets written to: model3/assets


INFO:tensorflow:Assets written to: model3/assets


In [ ]:
# model3 = tf.keras.models.load_model('model3/')

# Model4

In [69]:
model4 = Sequential()

model4.add(Embedding(input_dim=len(voc), output_dim=50, weights=[embeddings], trainable=True))

model4.add(Bidirectional(LSTM(128, return_sequences=True)))
model4.add(Bidirectional(LSTM(128, return_sequences=True)))
model4.add(Bidirectional(LSTM(128, return_sequences=True)))
model4.add(Bidirectional(LSTM(128, return_sequences=True)))
model4.add(Bidirectional(LSTM(128)))

model4.add(Dense(1, activation='sigmoid'))

model4.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model4.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 50)          2948450   
                                                                 
 bidirectional_1 (Bidirectio  (None, None, 256)        183296    
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, None, 256)        394240    
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, None, 256)        394240    
 nal)                                                            
                                                                 
 bidirectional_4 (Bidirectio  (None, None, 256)        394240    
 nal)                                                 

In [70]:
model4.fit(x_train_tmp, y_train_tmp, epochs=5, batch_size=32, verbose=2, use_multiprocessing=True)

Epoch 1/5
563/563 - 61s - loss: 0.6604 - accuracy: 0.5948 - 61s/epoch - 109ms/step
Epoch 2/5
563/563 - 49s - loss: 0.5543 - accuracy: 0.7132 - 49s/epoch - 88ms/step
Epoch 3/5
563/563 - 49s - loss: 0.4275 - accuracy: 0.8042 - 49s/epoch - 88ms/step
Epoch 4/5
563/563 - 49s - loss: 0.3104 - accuracy: 0.8701 - 49s/epoch - 87ms/step
Epoch 5/5
563/563 - 50s - loss: 0.2130 - accuracy: 0.9176 - 50s/epoch - 88ms/step


In [71]:
y_pred = model4.predict(x_test_tmp)
y_pred_tmp = np.round(y_pred)
y_pred_tmp = np.hstack(y_pred_tmp)
y_pred_tmp = y_pred_tmp.astype(np.int32)
print(classification_report(y_test, y_pred_tmp))

625/625 [==============================] - 26s 37ms/step
              precision    recall  f1-score   support

           0       0.92      0.95      0.93      9999
           1       0.95      0.92      0.93     10000

    accuracy                           0.93     19999
   macro avg       0.93      0.93      0.93     19999
weighted avg       0.93      0.93      0.93     19999



In [75]:
model4.save('model4/')

INFO:tensorflow:Assets written to: model4/assets


INFO:tensorflow:Assets written to: model4/assets


In [ ]:
# model4 = tf.keras.models.load_model('model4/')

# Transformer architecture models

In [27]:
bt = BertTokenizerFast.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [28]:
x_train = train['tweet'].copy()
y_train = train['label'].copy()

x_test = test['tweet'].copy()
y_test = test['label'].copy()

In [69]:
train_input_ids, train_attention_masks, train_outputs = [], [], []

In [70]:
for sentence, label in zip(x_train, y_train):
    
    sentence_tokens = bt.encode_plus(sentence, max_length=100, padding='max_length', truncation=True)
    
    train_input_ids.append(np.array(sentence_tokens['input_ids']))
    train_attention_masks.append(np.array(sentence_tokens['attention_mask']))
    train_outputs.append(label)

In [32]:
test_input_ids, test_attention_masks, test_outputs = [], [], []

In [33]:
for sentence, label in zip(x_test, y_test):
    
    sentence_tokens = bt.encode_plus(sentence, max_length=100, padding=True, truncation=True)
    
    test_input_ids.append(sentence_tokens['input_ids'])
    test_attention_masks.append(sentence_tokens['attention_mask'])
    test_outputs.append(label)

# Model1

In [37]:
bert_base_uncased = TFBertModel.from_pretrained('bert-base-uncased')

input_ids = Input(shape=(100,), name='input_ids', dtype='int32')
att_masks = Input(shape=(100,), name='masked_tokens', dtype='int32')

bert_in = bert_base_uncased(input_ids, attention_mask=att_masks)[1]

dense_1 = Dense(8, name='dense_1', activation=tf.keras.layers.LeakyReLU(alpha=.1))(bert_in)

out = Dense(1, name='output', activation='sigmoid')(dense_1)

bert_base_model1 = Model(inputs=[input_ids, att_masks], outputs=[out])

bert_base_model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [76]:
bert_base_model1.fit([np.array(train_input_ids), np.array(train_attention_masks)], np.array(train_outputs), epochs=1, batch_size=512, verbose=2, use_multiprocessing=True)

2022-06-20 17:17:29.293412: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 234.38MiB (rounded to 245760000)requested by op model/tf_bert_model_2/bert/encoder/layer_._0/attention/self/MatMul
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-06-20 17:17:29.293459: I tensorflow/core/common_runtime/bfc_allocator.cc:1027] BFCAllocator dump for GPU_0_bfc
2022-06-20 17:17:29.293471: I tensorflow/core/common_runtime/bfc_allocator.cc:1034] Bin (256): 	Total Chunks: 93, Chunks in use: 90. 23.2KiB allocated for chunks. 22.5KiB in use in bin. 580B client-requested in use in bin.
2022-06-20 17:17:29.293479: I tensorflow/core/common_runtime/bfc_allocator.cc:1034] Bin (512): 	Total Chunks: 1, Chunks in use: 1. 512B allocated for chunks. 512B in use in bin. 400B client-requeste

ResourceExhaustedError: Graph execution error:

Detected at node 'model/tf_bert_model_2/bert/encoder/layer_._0/attention/self/MatMul' defined at (most recent call last):
    File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/petar/jupyter/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/petar/jupyter/lib/python3.8/site-packages/traitlets/config/application.py", line 976, in launch_instance
      app.start()
    File "/home/petar/jupyter/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/petar/jupyter/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.8/asyncio/base_events.py", line 570, in r